In [37]:
import pandas as pd
import json

import sys
sys.path.append('../python')
import timeLimitEvents as timeLimits

In [38]:
dataEvents = pd.read_csv('E:/Documentos/PCEO/5/Informatica/TFG/datos/anonamyze_all_data_collection_v2.csv', sep=";")

dataEvents['time'] = pd.to_datetime(dataEvents['time'])
dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
dataEvents = dataEvents.sort_values('time')

In [39]:
typeMapping = ['Sandbox~SAND', '1. One Box~Tutorial', '2. Separated Boxes~Tutorial', '3. Rotate a Pyramid~Tutorial', '4. Match Silhouettes~Tutorial', '5. Removing Objects~Tutorial', '6. Stretch a Ramp~Tutorial', '7. Max 2 Boxes~Tutorial', '8. Combine 2 Ramps~Tutorial', '9. Scaling Round Objects~Tutorial',
               'Square Cross-Sections~Easy Puzzles', 'Bird Fez~Easy Puzzles', 'Pi Henge~Easy Puzzles', '45-Degree Rotations~Easy Puzzles',  'Pyramids are Strange~Easy Puzzles', 'Boxes Obscure Spheres~Easy Puzzles', 'Object Limits~Easy Puzzles', 'Not Bird~Easy Puzzles', 'Angled Silhouette~Easy Puzzles',
               'Warm Up~Hard Puzzles','Tetromino~Hard Puzzles', 'Stranger Shapes~Hard Puzzles', 'Sugar Cones~Hard Puzzles', 'Tall and Small~Hard Puzzles', 'Ramp Up and Can It~Hard Puzzles', 'More Than Meets Your Eye~Hard Puzzles', 'Unnecessary~Hard Puzzles', 'Zzz~Hard Puzzles', 'Bull Market~Hard Puzzles', 'Few Clues~Hard Puzzles', 'Orange Dance~Hard Puzzles', 'Bear Market~Hard Puzzles']

tutorialPuzzles = []

for puzzle in typeMapping:
    desc = puzzle.split("~")
    if(desc[1] == 'Tutorial'):
        tutorialPuzzles.append(desc[0])


intermediatePuzzles = []

for puzzle in typeMapping:
    desc = puzzle.split("~")
    if(desc[1] == 'Easy Puzzles'):
        intermediatePuzzles.append(desc[0])

advancedPuzzles = []

for puzzle in typeMapping:
    desc = puzzle.split("~")
    if(desc[1] == 'Hard Puzzles'):
        advancedPuzzles.append(desc[0])

In [40]:
def computeUserHistorial(dataEvents, user):

    completed = dict()
    tutorialCompleted = dict()
    intermediateCompleted = dict()
    advancedCompleted = dict()
    userAttempts = dict()
    userWins = dict()

    tutorialCompleted[user] = 0
    intermediateCompleted[user] = 0
    advancedCompleted[user] = 0
    userAttempts[user] = 0
    userWins[user] = 0

    puzzle_key = None

    for enum, event in dataEvents.iterrows():

        if event['user'] == user:

            if(event['type'] in ['ws-start_level', 'ws-puzzle_started']):

                puzzle_key =  json.loads(event['data'])['task_id']
                if(puzzle_key not in completed.keys()): completed[puzzle_key] = 0

            if(event['type'] == 'ws-puzzle_started' and completed[puzzle_key] == 0):

                userAttempts[user] +=1

            elif(event['type'] == 'ws-puzzle_complete' and completed[puzzle_key] == 0):

                userWins[user] += 1

                if (puzzle_key in tutorialPuzzles):
                    tutorialCompleted[user] += 1

                elif (puzzle_key in intermediatePuzzles):
                    intermediateCompleted[user] += 1

                elif (puzzle_key in advancedPuzzles):
                    advancedCompleted[user] += 1

                completed[puzzle_key] = 1

    resultado = dict()

    if userAttempts[user] == 0 or userWins[user] == 0:
        attempts_per_puzzle = 0
    else: attempts_per_puzzle = round((userAttempts[user]/userWins[user]),2)

    resultado['percentage_tutorial'] = round((tutorialCompleted[user]/9)*100,2)
    resultado['percentage_intermediate'] = round((intermediateCompleted[user]/9)*100,2)
    resultado['percentage_advanced'] = round((advancedCompleted[user]/13)*100,2)
    resultado['attempts_per_puzzle'] = attempts_per_puzzle

    return resultado

In [41]:
dataEvents1 = timeLimits.computeTimeLimitEvents(dataEvents,user_objective=['e129f5ced2d36f872763fd95b8747655'], timeLimit=pd.to_datetime('2019-09-10 13:40:02.862294-04:00',utc=True))
dataEvents2 = timeLimits.computeTimeLimitEvents(dataEvents,user_objective=['e129f5ced2d36f872763fd95b8747655'], timeLimit= pd.to_datetime('2019-09-10 13:58:03.173830-04:00',utc=True))

userHistorial1 = computeUserHistorial(dataEvents1, user = 'e129f5ced2d36f872763fd95b8747655' )
userHistorial2 = computeUserHistorial(dataEvents2, user = 'e129f5ced2d36f872763fd95b8747655')

In [42]:
print(userHistorial1)
print(userHistorial2)

{'percentage_tutorial': 0.0, 'percentage_intermediate': 11.11, 'percentage_advanced': 0.0, 'attempts_per_puzzle': 1.0}
{'percentage_tutorial': 0.0, 'percentage_intermediate': 33.33, 'percentage_advanced': 30.77, 'attempts_per_puzzle': 1.0}
